# PyPSA-USA Tutorial

In [ ]:
import pypsa
import os

In [ ]:
# Wildcards:
num_clusters = 20
interconnect = "texas"
opts = "Ep"
ll = "v1.0"
sector = "E"

run_name = "Tutorial"


resources_folder = os.path.join(
    os.path.dirname(os.getcwd()), "../", "resources", run_name
)
results_folder = os.path.join(os.path.dirname(os.getcwd()), "../", "results", run_name)


n_base_network = pypsa.Network(
    os.path.join(resources_folder, f"{interconnect}/elec_base_network.nc")
)
network_elec = pypsa.Network(
    os.path.join(resources_folder, f"{interconnect}/elec_base_network_l_pp.nc")
)
network_s = pypsa.Network(os.path.join(resources_folder, f"{interconnect}/elec_s.nc"))
network_c = pypsa.Network(
    os.path.join(resources_folder, f"{interconnect}/elec_s_{num_clusters}.nc")
)
network_c_ec = pypsa.Network(
    os.path.join(resources_folder, f"{interconnect}/elec_s_{num_clusters}_ec.nc")
)
# network_presolve = pypsa.Network(
#     os.path.join(
#         resources_folder,
#         f"{interconnect}/elec_s_{num_clusters}_ec_l{ll}_{opts}_{sector}.nc",
#     )
# )
# network_solved = pypsa.Network(
#     os.path.join(
#         results_folder,
#         f"{interconnect}/networks/elec_s_{num_clusters}_ec_l{ll}_{opts}_{sector}.nc",
#     )
# )

### Base Network
Reads in Breakthrough Energy/TAMU transmission dataset, and converts it into PyPSA compatible components. A base netowork file (`*.nc`) is written out. Included in this network are: Bus-Region memberships, Geolocated buses, Geoloactated AC and DC transmission lines + links, Transformers

In [ ]:
n_base_network.investment_periods

### Add Electricity
This module integrates data produced by `build_renewable_profiles`, `build_demand`, `build_cost_data`, `build_fuel_prices`, and `build_base_network` to create a network model that includes generators, demand, and costs. The module attaches generators, storage units, and loads to the network created by `build_base_network`. Each generator is assigned regional capital costs, and regional and daily or monthly marginal costs.

In [ ]:
network_elec.investment_periods

In [ ]:
network_elec.generators

### Simplified Network
Aggregates network to substations and simplifies to a single voltage level, and trims spur line-ends.

In [ ]:
network_s.loads_t.p_set

### Cluster Network
Creates networks clustered to ``{cluster}`` number of zones with aggregated
buses, generators and transmission corridors.

In [ ]:
network_c

In [ ]:
network_c.generators_t.p_max_pu.groupby(
    network_c.generators.carrier, axis=1
).mean().plot()

### Add Extra Components
Adds extendible components like battery energy storage, extendable conventional plants, and hydrogen infrastructure to the simplified network.


In [ ]:
network_c_ec

In [ ]:
network_c_ec.investment_periods

### Prepare Network
Prepare PyPSA network for solving according to `opts` and `ll`, such
as.
- adding an annual **limit** of carbon-dioxide emissions,
- adding an exogenous **price** per tonne emissions of carbon-dioxide (or other kinds),
- setting an **N-1 security margin** factor for transmission line capacities,
- specifying an expansion limit on the **cost** of transmission expansion,
- specifying an expansion limit on the **volume** of transmission expansion, and
- reducing the **temporal** resolution by averaging over multiple hours
  or segmenting time series into chunks of varying lengths using ``tsam``.

In [ ]:
network_presolve.global_constraints

In [ ]:
network_presolve.optimize(solver_name="gurobi")

In [ ]:
network_presolve.lines_t.p0.plot(legend=False)

In [ ]:
network_presolve.generators_t.p

### Solve Network

In [ ]:
network_solved